# Demo for AHA registry database 
This notebook demostrates the connection and query from the database.
<br />Demo for HF Tables.
<br />Created on December 1, 2022.
<br />@vihaan-manchanda

In [2]:
import psycopg2
import pandas as pd

## 1. Connection
Establish a connection to the database using psycopg2 client.

In [3]:
con = psycopg2.connect(host='ahadata-pmp.cr9jrs3l96oz.us-east-1.rds.amazonaws.com', 
                       database='ahadata_pmp', user='root', password='z3]<Be]jzMm4', port=3308)

## 2. Explore the database
Show all tables of each registry.
The table names and fields of each table will be included in a documentation.

*Note: it's a good practice to use try/except clause as it will remove the effect of any failed queries. Otherwise you'll encounter an "current transaction is aborted" error.*

In [4]:
cur = con.cursor()
try:
    cur.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name LIKE 'hf%'""")
    for table in cur.fetchall():
        print(table[0])
except:
    cur.rollback()

hf_hfdemographics
hf_hfdischarge
hf_hfmeta
hf_hfclinicalcodes
hf_hfadmin
hf_hfmeasures
hf_hfpatient
hf_hfsite
hf_hfinhospital
hf_hfadmission


In [5]:
# to view all the column names  in a particular table
cur.execute("Select * FROM hf_hfdemographics LIMIT 0")
colnames = [desc[0] for desc in cur.description]
colnames

['id',
 'gender',
 'dob',
 'dob_prec',
 'age',
 'zip',
 'paysource',
 'externalid',
 'race',
 'asian',
 'hisethni',
 'ethnicys',
 'hawaiian',
 'jc_homeless',
 'medicarejc',
 'patient_id',
 'agegroup',
 'patgenid',
 'patgenidoth',
 'patorien',
 'patorienoth',
 'agei',
 'genderi',
 'insurancei',
 'insurancei_new',
 'race2i',
 'racei',
 'jc_pmtsrce']

## 3. SQL Queries

In [6]:
# args: sql_query -> string sql query
# Returns a DataFrame corresponding to the result set of the query string

def create_pandas_table(sql_query, database = con):
    table = pd.read_sql_query(sql_query, database)
    return table

**3.1 Sample Query that outputs a table**

3.1 Use Case: It is useful to sometimes see what a query is returning in tabular form. In this example: We are querying the data to find the patients whose gender is Male.

In [7]:
sql_statement = """SELECT * from hf_hfdemographics
WHERE gender = '1'
"""

query1 =  create_pandas_table(sql_statement)
query1

,id,gender,dob,dob_prec,age,zip,paysource,externalid,race,asian,...,patgenidoth,patorien,patorienoth,agei,genderi,insurancei,insurancei_new,race2i,racei,jc_pmtsrce
0,807757,1,None,,86.0,,,,,,...,,,,86.0,0,,,5,7,
1,807758,1,None,,77.0,,,,3,,...,,,,77.0,0,,,1,5,
2,807760,1,None,,77.0,,,,3,,...,,,,77.0,0,3,3,3,1,5
3,807762,1,None,,63.0,97233,,,2,,...,,,,63.0,0,1,1,2,2,3
4,807765,1,None,,64.0,45013,,,3,,...,,,,64.0,0,4,4,1,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,817748,1,None,,70.0,18403,3,,3,,...,,,,70.0,0,,4,1,5,
5029,817750,1,None,,95.0,,,,3,,...,,,,95.0,0,3,3,1,5,1
5030,817752,1,None,,54.0,36619,,,4,,...,,,,54.0,0,4,4,4,4,4
5031,817755,1,None,,84.0,,,,3,,...,,,,84.0,0,2,2,1,5,1;2


**3.2 Get the total number of patients that are 'Male'**

In [8]:
sql_statement2 = """SELECT * from hf_hfdemographics
WHERE gender = '1'
"""
table2 =  create_pandas_table(sql_statement2)
print("Total Number of Male Patients are:", len(table2))

Total Number of Male Patients are: 5033


**3.3 Complicated Queries** 

AHAHF6 : Percentage of patients with heart failure who are prescribed an ARNI at hospital discharge(from quality measure doc)

In [66]:
sql_statement3 = """SELECT * from hf_hfinhospital 
INNER JOIN hf_hfadmission
ON hf_hfinhospital.id =  hf_hfadmission.id
INNER JOIN hf_hfdischarge
ON hf_hfinhospital.id =   hf_hfdischarge.id
WHERE hfdiagnosis = '4' or hfdiagnosis = '130'
and (lvfasmt <= '40' or lvfqual = '3')
and (dschstat <> '2' and dschstat <> '3'  and dschstat <> '4'  and dschstat <> '6'  and dschstat <> '7')
and (presarni <> '2' and presarni <> '3')
and (presarb <> '2' and presarb <> '3')
and (arnireason <> '1')
and (comfortonly <> '1' and comfortonly <> '2' and comfortonly <> '3')
and medhist <> '42'
and hfprocedures <> '120' 
"""
denominator_table_3 =  create_pandas_table(sql_statement3)
denominator_3 = len(denominator_table_3)
print("Denominator is:", denominator_3)


Denominator is: 5580


In [62]:
# Note, TODO: cast column hrdsch to integer.

numerator_5_sql = """SELECT *
from hf_hfdischarge

WHERE presarni = '1'

"""
numerator5_query_table =  create_pandas_table(numerator_5_sql)
numerator_5 = len(numerator5_query_table)

print("Numerator is:", numerator_5)

Numerator is: 257


In [65]:
percentage_5 = (numerator_5/denominator_3)*100
print(percentage_5)

4.60573476702509


## 4. Genral Advice for writing SQL queries

1. The first step is to figure out what is the corresponding variable name to the verbose name, and also identifyig the table in which this variable exists.
2. To get the variable name one can use a combination of verbose/ human readable column names and compare that with the the variable name which can be found in the meta. An alternative is to look at the config file for the specific registry which is provided.
3. Understand the type of join you want to use. Inner joins can be really helpful.
4. The registry specific quality measure document is a good strating point to write queries.